In [2]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [3]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F

In [4]:
from transformers import Wav2Vec2Processor, AutoModelForCTC

In [5]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer

# info


https://github.com/NVIDIA/NeMo/blob/stable/examples/slu/speech_intent_slot/run_speech_intent_slot_train.py

In [56]:
config_dir = '/home/evgenshuben/Desktop/gitReps/YandexCup/configs'
config_name = 'config'


with initialize_config_dir(config_dir=config_dir, version_base=None):
    cfg = compose(config_name=config_name)
    # cfg['dataset']['train']['train']['dataset_path']

[NeMo W 2024-10-31 20:17:16 nemo_logging:349] /home/evgenshuben/miniconda3/envs/DL24/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
      warnings.warn(msg, UserWarning)
    


In [57]:
cfg.enviroment.device = 'cpu'

In [58]:
trainer = Trainer(cfg)

[NeMo I 2024-10-31 20:17:22 cloud:58] Found existing object /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.
[NeMo I 2024-10-31 20:17:22 cloud:64] Re-using file from: /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2024-10-31 20:17:22 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-10-31 20:17:22 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-10-31 20:17:23 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-10-31 20:17:23 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-10-31 20:17:23 features:289] PADDING: 0


[NeMo W 2024-10-31 20:17:23 nemo_logging:349] /home/evgenshuben/miniconda3/envs/DL24/lib/python3.10/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues r

[NeMo I 2024-10-31 20:17:23 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


In [59]:
for batch in trainer.train_dataloader:
    break

In [64]:
trainer.cfg.num_classes

39535

In [61]:
batch['anchor'].shape

torch.Size([64, 84, 50])

In [73]:
import numpy as np

In [84]:
class MixUp:
    def __init__(self, num_classes, alpha=1.0):
        self.num_classes = num_classes
        self.alpha = alpha

    def __call__(self, x, y):
        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
        else:
            lam = 1

        batch_size = x.size(0)
        index = torch.randperm(batch_size)

        mixed_x = lam * x + (1 - lam) * x[index, :]

        # Преобразование y в one-hot векторы
        y_one_hot = torch.zeros(batch_size, self.num_classes, device=x.device)
        y_one_hot[range(batch_size), y] = 1  # Преобразуем в one-hot

        mixed_y = lam * y_one_hot + (1 - lam) * y_one_hot[index]

        return mixed_x, mixed_y


class CutMix:
    def __init__(self, num_classes, alpha=1.0):
        self.num_classes = num_classes
        self.alpha = alpha

    def __call__(self, x, y):
        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
        else:
            lam = 1

        batch_size = x.size(0)
        index = torch.randperm(batch_size)

        W = x.size(2)
        H = x.size(1)

        r_x = np.random.randint(0, W)
        r_y = np.random.randint(0, H)
        r_w = int(W * np.sqrt(1 - lam))
        r_h = int(H * np.sqrt(1 - lam))

        r_x1 = np.clip(r_x - r_w // 2, 0, W)
        r_x2 = np.clip(r_x + r_w // 2, 0, W)
        r_y1 = np.clip(r_y - r_h // 2, 0, H)
        r_y2 = np.clip(r_y + r_h // 2, 0, H)

        # Создаем смешанные данные
        x[:, r_y1:r_y2, r_x1:r_x2] = x[index, r_y1:r_y2, r_x1:r_x2]

        # Преобразование y в one-hot векторы
        y_one_hot = torch.zeros(batch_size, self.num_classes, device=x.device)
        y_one_hot[range(batch_size), y] = 1  # Преобразуем в one-hot

        # Смешиваем метки
        y_a = y_one_hot
        y_b = y_one_hot[index]
        mixed_y = lam * y_a + (1 - lam) * y_b

        return x, mixed_y

In [75]:
class AugmentationCollator:
    def __init__(self, augmentations):
        self.augmentations = augmentations

    def __call__(self, batch):

        anchor_ids = [item['anchor_id'] for item in batch]  # ID без изменений
        anchors = torch.stack([item['anchor'] for item in batch])  # Тензор изображений
        labels = torch.stack([item['anchor_label'] for item in batch])  # Тензор меток

        augmented_anchors, augmented_labels = self.augmentations(anchors, labels.long())

        return {
            'anchor_id': anchor_ids,
            'anchor': augmented_anchors,
            'anchor_label': augmented_labels
        }

In [102]:
from torchvision.transforms.v2 import RandomChoice

In [97]:
from torch.utils.data import DataLoader

In [103]:
augmentations = RandomChoice([
            MixUp(num_classes=num_classes),
            CutMix(num_classes=num_classes)
        ])

In [104]:
collator = AugmentationCollator(augmentations=augmentations)

In [105]:
trainer.train_dataloader.dataset[0]['anchor'].shape

torch.Size([84, 50])

In [106]:
data_loader = DataLoader(trainer.train_dataloader.dataset, batch_size=32, collate_fn=collator)

In [107]:
for batch in data_loader:
    break

In [95]:
dataloader = trainer.train_dataloader.dataset

In [96]:
DataLoader

NameError: name 'DataLoader' is not defined

In [85]:
num_classes = trainer.cfg.num_classes

In [86]:
anchors = batch['anchor']
labels = batch['anchor_label']

In [92]:
augmentations[0](anchors, labels.long())[0]

tensor([[[-27.4922, -24.0988, -18.5980,  ..., -25.0063, -23.5181, -22.5378],
         [-17.8997, -19.3798, -20.3946,  ..., -29.3251, -34.3992, -29.5529],
         [-20.1813, -19.8249, -19.6386,  ..., -27.8654, -26.7650, -25.3741],
         ...,
         [-28.4069, -27.6728, -35.5562,  ..., -31.4960, -28.2954, -27.0652],
         [-25.7478, -26.3863, -27.0308,  ..., -20.7767, -22.2248, -24.9296],
         [-30.0559, -24.8653, -30.4569,  ..., -25.3193, -24.2758, -25.0753]],

        [[-49.4972, -47.4558, -48.4513,  ..., -37.9406, -29.7031, -25.1541],
         [-47.4412, -51.0624, -47.0224,  ..., -38.8281, -34.4013, -29.2891],
         [-39.1572, -38.4866, -42.7885,  ..., -31.9471, -32.9739, -29.4244],
         ...,
         [-35.7585, -37.4499, -25.9650,  ..., -17.9286, -15.8398, -14.8191],
         [-42.6736, -39.8472, -33.4855,  ..., -20.8357, -21.0604, -27.2990],
         [-39.7250, -42.1776, -34.2852,  ..., -24.2008, -23.2768, -28.7641]],

        [[-27.8135, -28.4518, -31.7045,  ...

In [200]:
specrtogramm = trainer.train_dataloader.dataset[0]['anchor']

In [201]:
specrtogramm.shape

torch.Size([80, 50])

In [202]:
for batch in trainer.train_dataloader:
    break

In [203]:
batch['anchor'].shape

torch.Size([64, 80, 50])

In [163]:
encoder = asr_model.encoder

In [164]:
mel_spectrograms = batch['anchor']


In [165]:
mel_spectrograms.shape

torch.Size([64, 80, 50])

In [204]:
from nemo.collections.asr.models import ASRModel

In [211]:
pretrained_encoder_name

'stt_en_conformer_ctc_small'

# линейный слой для интерполяции

In [23]:
for batch in trainer.train_dataloader:
    break

In [24]:
batch['anchor'].shape

torch.Size([64, 84, 50])

In [48]:
x = batch['anchor']

batch_size, freq_size, time_size = x.size()
x = x.permute(0, 2, 1).reshape(-1, freq_size)  # Приводим к (batch_size * time_size, 84)

In [49]:
freq_transform = nn.Linear(84, 80)

In [54]:
x = freq_transform(x)
x.shape

torch.Size([3200, 80])

In [51]:
batch_size

64

In [55]:
x.view(batch_size, time_size, -1).shape

torch.Size([64, 50, 80])

In [40]:
x = x.view(batch_size, time_size, -1).permute(0, 2, 1)  # Теперь (batch_size, 80, 50)

In [41]:
x.shape

torch.Size([64, 84, 50])

In [283]:
class AudioConformer(nn.Module):
    def __init__(
        self,
        pretrained_model_name='stt_en_conformer_ctc_small',
        emb_size=512,
        num_classes=10
    ):
        super().__init__()

        self.pretrained_model = ASRModel.from_pretrained(
            model_name=pretrained_model_name,
            # map_location=torch.device("cuda")
        )

        self.encoder = self.pretrained_model.encoder
        
        self.embedding_size = self.encoder.pre_encode.out.out_features
        
        self.fc_emb = nn.Linear(self.embedding_size, emb_size) if emb_size != self.embedding_size else nn.Identity()

        self.clf = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        encoder_output, lengths = self.encoder(
            audio_signal=x,
            length=torch.tensor([x.shape[2]] * x.shape[0], device=x.device)  # Убедитесь, что длины находятся на том же устройстве
        )
        
        features = encoder_output.mean(dim=2)  # Усредняем по временной оси
        emb = self.fc_emb(features)

        cls = self.clf(emb)
        return dict(emb=emb, cls=cls)

In [284]:
x = batch['anchor']

In [285]:
torch.tensor([x.shape[2]] * x.shape[0], device=x.device)

tensor([50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50])

In [286]:
model = AudioConformer()

[NeMo I 2024-10-31 19:11:40 cloud:58] Found existing object /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.
[NeMo I 2024-10-31 19:11:40 cloud:64] Re-using file from: /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2024-10-31 19:11:40 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-10-31 19:11:41 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-10-31 19:11:41 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-10-31 19:11:41 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-10-31 19:11:41 features:289] PADDING: 0
[NeMo I 2024-10-31 19:11:41 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


In [287]:
model.to('cuda')

AudioConformer(
  (pretrained_model): EncDecCTCModelBPE(
    (preprocessor): AudioToMelSpectrogramPreprocessor(
      (featurizer): FilterbankFeatures()
    )
    (encoder): ConformerEncoder(
      (pre_encode): ConvSubsampling(
        (out): Linear(in_features=3520, out_features=176, bias=True)
        (conv): Sequential(
          (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (3): ReLU(inplace=True)
        )
      )
      (pos_enc): RelPositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layers): ModuleList(
        (0-15): 16 x ConformerLayer(
          (norm_feed_forward1): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
          (feed_forward1): ConformerFeedForward(
            (linear1): Linear(in_features=176, out_features=704, bias=True)
            (activation): Swish()
            (drop

In [288]:
model(x.to('cuda'))['emb'].shape

torch.Size([64, 512])

In [295]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [296]:
num_params

13249787

In [14]:
import torch
from torch import nn
from nemo.collections.asr.models import ASRModel

In [15]:
pretrained_encoder_name = 'stt_en_conformer_ctc_small'

In [16]:
model_cls = ASRModel

pretraind_model = model_cls.from_pretrained(
    model_name=pretrained_encoder_name, map_location=torch.device("cpu")
)

encoder = pretraind_model.encoder

[NeMo I 2024-10-31 19:52:46 cloud:58] Found existing object /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.
[NeMo I 2024-10-31 19:52:46 cloud:64] Re-using file from: /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2024-10-31 19:52:46 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-10-31 19:52:47 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-10-31 19:52:47 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-10-31 19:52:47 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-10-31 19:52:47 features:289] PADDING: 0


[NeMo W 2024-10-31 19:52:47 nemo_logging:349] /home/evgenshuben/miniconda3/envs/DL24/lib/python3.10/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues r

[NeMo I 2024-10-31 19:52:47 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


In [17]:
encoder

ConformerEncoder(
  (pre_encode): ConvSubsampling(
    (out): Linear(in_features=3520, out_features=176, bias=True)
    (conv): Sequential(
      (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (pos_enc): RelPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (layers): ModuleList(
    (0-15): 16 x ConformerLayer(
      (norm_feed_forward1): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
      (feed_forward1): ConformerFeedForward(
        (linear1): Linear(in_features=176, out_features=704, bias=True)
        (activation): Swish()
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=704, out_features=176, bias=True)
      )
      (norm_conv): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
      (conv): ConformerConvolution(
        (pointwise_conv1)

In [ ]:
encoder()

In [212]:
mel_spectrograms.shape[0]

32

In [213]:
encoder_output = encoder(audio_signal=mel_spectrograms, length=torch.tensor([mel_spectrograms.shape[2]] * mel_spectrograms.shape[0]))

In [216]:
encoder_output[0].shape

torch.Size([32, 176, 13])

In [217]:
encoder_output[1].shape

torch.Size([32])

In [34]:
# encoder = asr_model.encoder()

In [32]:
# asr_model.eval()

In [33]:
# encoder = asr_model.encoder()

In [29]:
from nemo.collections.asr.modules import ConformerEncoder

In [48]:
# encoder = asr_model.encoder(
#     feat_in=80,
#     n_layers=18,
#     d_model=512,
#     feat_out=-1
# )

In [40]:
encoder = asr_model.encoder

In [168]:
mel_spectrograms = batch['anchor']

In [169]:
mel_spectrograms.shape

torch.Size([64, 80, 50])

In [170]:
encoder_output = encoder(audio_signal=mel_spectrograms, length=torch.tensor([mel_spectrograms.shape[2]] * 64))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [14]:
input_signal = batch['anchor']
signal_length = torch.tensor([input_signal.shape[-1]] * input_signal.shape[0])

# nvidia tutrial

In [171]:
from nemo.collections.asr.models import ASRModel, SLUIntentSlotBPEModel, SpeechEncDecSelfSupervisedModel

In [184]:
pretrained_encoder_name = 'stt_en_conformer_ctc_large'
pretrained_encoder_name = 'stt_en_conformer_ctc_small'

In [185]:
model_cls = ASRModel

In [186]:
pretraind_model = model_cls.from_pretrained(
    model_name=pretrained_encoder_name, map_location=torch.device("cpu")
)

[NeMo I 2024-10-31 17:32:00 cloud:58] Found existing object /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.
[NeMo I 2024-10-31 17:32:00 cloud:64] Re-using file from: /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2024-10-31 17:32:00 common:924] Instantiating model from pre-trained checkpoint
[NeMo I 2024-10-31 17:32:01 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-10-31 17:32:01 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-10-31 17:32:01 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-10-31 17:32:01 features:289] PADDING: 0
[NeMo I 2024-10-31 17:32:01 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/evgenshuben/.cache/torch/NeMo/NeMo_1.23.0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


In [187]:
encoder = pretraind_model.encoder

In [188]:
encoder

ConformerEncoder(
  (pre_encode): ConvSubsampling(
    (out): Linear(in_features=3520, out_features=176, bias=True)
    (conv): Sequential(
      (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (pos_enc): RelPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (layers): ModuleList(
    (0-15): 16 x ConformerLayer(
      (norm_feed_forward1): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
      (feed_forward1): ConformerFeedForward(
        (linear1): Linear(in_features=176, out_features=704, bias=True)
        (activation): Swish()
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=704, out_features=176, bias=True)
      )
      (norm_conv): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
      (conv): ConformerConvolution(
        (pointwise_conv1)

In [189]:
batch['anchor'].shape

torch.Size([64, 80, 50])

In [190]:
processed_signal = batch['anchor']

In [191]:
mel_spectrograms = batch['anchor']

In [192]:
mel_spectrograms.shape[2]

50

In [193]:
mel_spectrograms.shape

torch.Size([64, 80, 50])

In [194]:
batch_size = 32
freq = 80  # Уменьшим размер частот для совместимости
time = 50  # Увеличим временной размер для совместимости

# Инициализация случайных данных для примера
mel_spectrograms = torch.rand(batch_size, freq, time)

In [195]:
mel_spectrograms.shape

torch.Size([32, 80, 50])

In [197]:
encoder_output = encoder(audio_signal=mel_spectrograms, length=torch.tensor([mel_spectrograms.shape[2]] * batch_size))

In [198]:
encoder_output[0].shape

torch.Size([32, 176, 13])

In [ ]:
encoder(audio_signal=processed_signal, length=processed_signal_len)

In [58]:
encoder(audio_signal=processed_signal, length=50)

AttributeError: 'int' object has no attribute 'to'

In [59]:
length=torch.tensor([processed_signal.shape[3]] * processed_signal)

IndexError: tuple index out of range

In [60]:
processed_signal.shape

torch.Size([64, 84, 50])